# Example code of memory in LangChain

LLMs are stateless, which means they can't remember any previous information when they run. You get round this by giving them access to 'memory', or the results of previous steps. Here's some code which shows how you can implement this

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import OllamaLLM


# Set up the LLM
llm = OllamaLLM(model="gemma3:1b", temperature=0.6)

# Prompt template with memory placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# Compose into a chain
chain = prompt | llm

# Create a simple session store using a dictionary
session_histories = {}

# Function to get or create message history for each session
def get_message_history(session_id: str):
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()
    return session_histories[session_id]

# Wrap chain in memory-aware runnable
with_history = RunnableWithMessageHistory(
    chain,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

# Simulate a session
session_id = "alistair's-session"

# Interact with the chain
response_1 = with_history.invoke(
    {"input": "Hey there - I'm Alistair, how are you doing today?"},
    config={"configurable": {"session_id": session_id}}
)

response_2 = with_history.invoke(
    {"input": "Where does the wind come from?"},
    config={"configurable": {"session_id": session_id}}
)

response_3 = with_history.invoke(
    {"input": "What's my name?"},
    config={"configurable": {"session_id": session_id}}
)

# Print results
print(response_1)
print(response_2)
print(response_3)

# Optional: see memory contents
print(f"\nMemory for session '{session_id}':")
for msg in session_histories[session_id].messages:
    print(f"- {msg.type}: {msg.content}")


Hey there, Alistair! I’m doing well, thanks for asking. Just here, ready to assist. How about you? How’s your day going so far?
AI: That’s a fantastic question! The wind comes from all directions, Alistair. It’s a complex phenomenon driven by a few key factors. Essentially, it’s caused by differences in temperature and pressure. 

Here’s a breakdown of how it works:

* **Warm Air Rises:** Warm air is less dense than cooler air, so it naturally rises. This is a fundamental principle of weather.
* **Uneven Heating:** The Earth's surface isn't uniform. It warms up faster at the equator and cools down faster at the poles. This creates areas of higher and lower atmospheric pressure.
* **Low-Pressure Zones:** Areas of low pressure create an area of rising air.  These low-pressure zones are where winds are most easily generated.
* **Rotation:** Earth’s rotation plays a huge role! The Coriolis effect, caused by the Earth's spin, deflects winds to the right in the Northern Hemisphere and to the